# Part 2: Baseball Data

*Introductory - Intermediate level SQL*

---

## Setup

`cd` into the directory you'd like to use for this challenge. Then, download the Lahman SQL Lite dataset

```
curl -L -o lahman.sqlite https://github.com/WebucatorTraining/lahman-baseball-mysql/raw/master/lahmansbaseballdb.sqlite
```

*The `-L` follows redirects, and the `-o` uses the filename instead of outputting to the terminal.*

Make sure sqlite3 is installed

```
conda install -c anaconda sqlite
```

In your notebook, check out the schema

In [3]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('../../baseball_data/lahman.sqlite')

query = "SELECT * FROM sqlite_master;"

df_schema = pd.read_sql_query(query, conn)

df_schema.tbl_name.unique()

array(['allstarfull', 'appearances', 'awardsmanagers', 'awardsplayers',
       'awardssharemanagers', 'awardsshareplayers', 'batting',
       'battingpost', 'collegeplaying', 'divisions', 'fielding',
       'fieldingof', 'fieldingofsplit', 'fieldingpost', 'halloffame',
       'homegames', 'leagues', 'managers', 'managershalf', 'parks',
       'people', 'pitching', 'pitchingpost', 'salaries', 'schools',
       'seriespost', 'teams', 'teamsfranchises', 'teamshalf'],
      dtype=object)

Please complete this exercise using SQL Lite (i.e., the Lahman baseball data, above) and your Jupyter notebook.

1. What was the total spent on salaries by each team, each year?

In [17]:
query = '''
SELECT teamID, yearID, SUM(salary) AS totalSalaries
FROM salaries
GROUP BY teamID, yearID;
'''

df = pd.read_sql_query(query, conn)
df.head()

,teamID,yearID,totalSalaries
0,ATL,1985,14807000.0
1,BAL,1985,11560712.0
2,BOS,1985,10897560.0
3,CAL,1985,14427894.0
4,CHA,1985,9846178.0


2. What is the first and last year played for each player? *Hint:* Create a new table from 'Fielding.csv'.

In [18]:
query = '''
SELECT playerID, MIN(yearID) AS firstYear, MAX(yearID) AS lastYear
FROM fielding
GROUP BY playerID;
'''

df = pd.read_sql_query(query, conn)
df.head()

,playerID,firstYear,lastYear
0,aardsda01,2004,2015
1,aaronha01,1954,1976
2,aaronto01,1962,1971
3,aasedo01,1977,1990
4,abadan01,2001,2003


3. Who has played the most all star games?

In [38]:
query = '''
SELECT nameGiven AS name, COUNT(gameID) AS games
FROM allstarfull
JOIN people ON allstarfull.playerID = people.playerID
GROUP BY allstarfull.playerID
ORDER BY COUNT(*) DESC;
'''

df = pd.read_sql_query(query, conn)
df.head()

,name,games
0,Henry Louis,24
1,Willie Howard,24
2,Stanley Frank,24
3,Mickey Charles,19
4,Calvin Edwin,19


4. Which school has generated the most distinct players? *Hint:* Create new table from 'CollegePlaying.csv'.

In [58]:
query = '''
SELECT name_full AS schoolName, COUNT(DISTINCT(playerID)) AS distinctPlayers
FROM collegeplaying
JOIN schools ON collegeplaying.schoolID = schools.schoolID
GROUP BY collegeplaying.schoolID
ORDER BY COUNT(DISTINCT(playerID)) DESC
LIMIT 1
'''
df = pd.read_sql_query(query, conn)
df.head()

,schoolName,distinctPlayers
0,University of Texas at Austin,107


5. Which players have the longest career? Assume that the `debut` and `finalGame` columns comprise the start and end, respectively, of a player's career. *Hint:* Create a new table from 'Master.csv'. Also note that strings can be converted to dates using the [`DATE`](https://wiki.postgresql.org/wiki/Working_with_Dates_and_Times_in_PostgreSQL#WORKING_with_DATETIME.2C_DATE.2C_and_INTERVAL_VALUES) function and can then be subtracted from each other yielding their difference in days.

In [69]:
query = '''
SELECT nameGiven, (DATE(finalGame) - DATE(debut)) AS careerLength 
FROM people
ORDER BY careerLength DESC
'''
df = pd.read_sql_query(query, conn)
df.head()

,nameGiven,careerLength
0,Nicholas,35.0
1,James Henry,32.0
2,Saturnino Orestes Armas,31.0
3,Charles Timothy,30.0
4,Walter Arlington,29.0


6. What is the distribution of debut months? *Hint:* Look at the `DATE` and [`EXTRACT`](https://www.postgresql.org/docs/current/static/functions-datetime.html#FUNCTIONS-DATETIME-EXTRACT) functions.

In [133]:
query = '''
SELECT DATE(debut), COUNT(nameGiven)
FROM people
GROUP BY DATE(debut)
'''
df = pd.read_sql_query(query, conn)
df.head()

,DATE(debut),COUNT(nameGiven)
0,None,196
1,1871-05-04,18
2,1871-05-05,19
3,1871-05-06,9
4,1871-05-08,9


7. What is the effect of table join order on mean salary for the players listed in the main (master) table? *Hint:* Perform two different queries, one that joins on playerID in the salary table and other that joins on the same column in the master table. You will have to use left joins for each since right joins are not currently supported with SQLalchemy.

In [127]:
query = '''
SELECT nameGiven, AVG(salary)
FROM people
LEFT JOIN salaries ON people.playerID = salaries.playerID
GROUP BY people.playerID
'''
df = pd.read_sql_query(query, conn)
df.head()

,nameGiven,AVG(salary)
0,David Allan,1.322821e+06
1,Henry Louis,NaN
2,Tommie Lee,NaN
3,Donald William,5.750000e+05
4,Fausto Andres,3.270000e+05


In [128]:
query = '''
SELECT nameGiven, AVG(salary)
FROM salaries
JOIN salaries ON people.playerID = salaries.playerID
GROUP BY people.playerID
'''
df = pd.read_sql_query(query, conn)
df.head()

,nameGiven,AVG(salary)
0,David Allan,1.322821e+06
1,Donald William,5.750000e+05
2,Fausto Andres,3.270000e+05
3,Fernando Antonio,7.532800e+05
4,Jeffrey William,2.462500e+05
